<a href="https://colab.research.google.com/github/BunnyNoBugs/ParaCAIF/blob/main/toxicity_classification/bert_toxicity_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets huggingface_hub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
!wget https://raw.githubusercontent.com/BunnyNoBugs/russian-detoxification/main/data/toxicity_classification/classification_train.csv
!wget https://raw.githubusercontent.com/BunnyNoBugs/russian-detoxification/main/data/toxicity_classification/classification_dev.csv

--2023-04-25 21:21:29--  https://raw.githubusercontent.com/BunnyNoBugs/russian-detoxification/main/data/toxicity_classification/classification_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1576950 (1.5M) [text/plain]
Saving to: ‘classification_train.csv’

classification_trai 100%[===================>]   1.50M  --.-KB/s    in 0.02s   

2023-04-25 21:21:29 (90.1 MB/s) - ‘classification_train.csv’ saved [1576950/1576950]

--2023-04-25 21:21:29--  https://raw.githubusercontent.com/BunnyNoBugs/russian-detoxification/main/data/toxicity_classification/classification_dev.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuserconte

In [ ]:
#global constants

checkpoint = 'cointegrated/rubert-tiny2'
task_name = 'russe-toxicity'

In [ ]:
train_df = pd.read_csv("classification_train.csv")
dev_df = pd.read_csv("classification_dev.csv")

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 13896
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
id2label = {0: 'neutral', 1: 'toxic'}
label2id = {'neutral': 0, 'toxic': 1}

In [ ]:
def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples['text'], truncation=True)
    tokenized_inputs['label'] = [label2id[x] for x in examples['label']]
    
    return tokenized_inputs

In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_dev_dataset = dev_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/13896 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('BunnyNoBugs/rubert-tiny2-russe-toxicity')

In [ ]:
model_name = checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-{task_name}",
    evaluation_strategy = "steps",
    logging_steps=100,
    save_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=1,
    save_steps=100,
    metric_for_best_model='f1_macro',
    load_best_model_at_end=True,
    # push_to_hub=True
)

In [ ]:
def compute_metrics(p):
    outputs, labels = p
    predictions = np.argmax(outputs, axis=1)

    precision_macro = precision_score(labels, predictions, average='macro')
    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_classes = f1_score(labels, predictions, average=None)
    f1_classes = {f'f1_{cls_name}': f1 for cls_name, f1 in zip(label2id, f1_classes)}

    metrics = {
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
    }
    metrics.update(f1_classes)

    return metrics

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.20180724561214447,
 'eval_precision_macro': 0.9412941294129413,
 'eval_recall_macro': 0.94125,
 'eval_f1_macro': 0.9412485312132803,
 'eval_f1_neutral': 0.941542288557214,
 'eval_f1_toxic': 0.9409547738693467,
 'eval_runtime': 4.3417,
 'eval_samples_per_second': 368.522,
 'eval_steps_per_second': 5.758}

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,F1 Neutral,F1 Toxic
100,0.612700,0.447669,0.825246,0.825000,0.824967,0.827374,0.822560
200,0.372800,0.285199,0.887655,0.887500,0.887489,0.888614,0.886364
300,0.288800,0.240468,0.903912,0.903750,0.903740,0.904703,0.902778
400,0.272400,0.228821,0.911046,0.909375,0.909283,0.912174,0.906391
500,0.238700,0.216117,0.913246,0.912500,0.912460,0.914321,0.910600
600,0.218900,0.209785,0.920631,0.920625,0.920625,0.920774,0.920476
700,0.178300,0.204462,0.923274,0.923125,0.923118,0.923839,0.922397
800,0.191100,0.209757,0.918444,0.917500,0.917453,0.915493,0.919414
900,0.174900,0.194729,0.927116,0.926875,0.926865,0.927733,0.925996
1000,0.172800,0.189264,0.926274,0.926250,0.926249,0.926526,0.925972


TrainOutput(global_step=4350, training_loss=0.14328423867280457, metrics={'train_runtime': 669.7426, 'train_samples_per_second': 207.483, 'train_steps_per_second': 6.495, 'total_flos': 103589287717344.0, 'train_loss': 0.14328423867280457, 'epoch': 10.0})

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Apr24_17-13-06_a6fbd89319ff/events.out.tfevents.1682356391.a6fbd89319ff.5571.0:   0%|        …

To https://huggingface.co/BunnyNoBugs/rubert-tiny2-russe-toxicity
   9e807bb..2b314a1  main -> main

   9e807bb..2b314a1  main -> main

To https://huggingface.co/BunnyNoBugs/rubert-tiny2-russe-toxicity
   2b314a1..b136cda  main -> main

   2b314a1..b136cda  main -> main



'https://huggingface.co/BunnyNoBugs/rubert-tiny2-russe-toxicity/commit/2b314a1d6711b3fdc2f8ba1061bde31a23a3b073'

In [ ]:
trainer.evaluate()

{'eval_loss': 0.20180724561214447,
 'eval_precision_macro': 0.9412941294129413,
 'eval_recall_macro': 0.94125,
 'eval_f1_macro': 0.9412485312132803,
 'eval_f1_neutral': 0.941542288557214,
 'eval_f1_toxic': 0.9409547738693467,
 'eval_runtime': 0.3379,
 'eval_samples_per_second': 4735.628,
 'eval_steps_per_second': 73.994,
 'epoch': 10.0}